In [1]:
# This snippet of code properly adds the working source root path to python's path
# so you no longer have to install spykshrk through setuptools
import sys, os
root_depth = 2
notebook_dir = globals()['_dh'][0]
root_path = os.path.abspath(os.path.join(notebook_dir, '../'*root_depth))
# Add to python's path
try:
    while True:
        sys.path.remove(root_path)
except ValueError:
    # no more root paths
    pass
sys.path.append(root_path)
# Alternatively set root path as current working directory
#os.chdir(root_path)

In [3]:
import scipy as sp
import numpy as np
import pandas as pd
from spykshrk.franklab.data_containers import FlatLinearPosition, SpikeFeatures, \
        EncodeSettings, pos_col_format, SpikeObservation
from spykshrk.franklab.pp_decoder.util import normal_pdf_int_lookup, gaussian
from spykshrk.util import AttrDict
import holoviews as hv
hv.extension('bokeh', 'matplotlib')

In [4]:
encode_settings = AttrDict(pos_kernel_std=2, mark_kernel_std=10, pos_bins=np.arange(0, 40, 1))

spk_time = [1, 2, 3, 10, 11]
spk_marks = [[30, 30, 30, 30], 
             [40, 40, 40, 40], 
             [35, 35, 35, 35],
             [30, 30, 30, 30],
             [31, 31, 31, 31]]
mark_columns = ['c00', 'c01', 'c02', 'c03']
linpos_flat_obj = pd.DataFrame((1, 2, 3, 30, 31), index=spk_time, columns=['linpos_flat'])
enc_spk = pd.DataFrame(spk_marks, index=spk_time, columns=mark_columns)
dec_spk = pd.DataFrame(spk_marks, index=spk_time, columns=mark_columns)


In [5]:
pos_distrib_tet = sp.stats.norm.pdf(np.expand_dims(encode_settings.pos_bins, 0),
                                    np.expand_dims(linpos_flat_obj['linpos_flat'], 1),
                                    encode_settings.pos_kernel_std)
mark_contrib = normal_pdf_int_lookup(np.expand_dims(dec_spk[mark_columns], 1),
                                     np.expand_dims(enc_spk, 0),
                                     encode_settings.mark_kernel_std)
all_contrib = np.prod(mark_contrib, axis=2)
observ = np.matmul(all_contrib, pos_distrib_tet)

In [6]:
mark_contrib.shape

In [16]:
observ.shape

In [55]:
hv.Image(pos_distrib_tet, bounds=[encode_settings.pos_bins[0], 0, encode_settings.pos_bins[-1], len(spk_marks)])

In [52]:
len(spk_marks)